In [ ]:
#import the needed libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os
import cv2
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split
import tensorflow 
from tensorflow import keras
from keras.utils.np_utils import to_categorical
from sklearn.metrics import confusion_matrix
from keras.models import Sequential,load_model,Model
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D,ZeroPadding2D,Activation
from tensorflow.keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten,Input
import itertools

In [ ]:
#
path_folder=['test']
result=[]
for i in path_folder:    
    path=f'C:\\Users\\svnduw\\Desktop\\Bekbol\\Bekbol project\\age detection\\20-50\{i}'
    folders=os.listdir(path)
    df=pd.DataFrame(columns=['type','link','age'])
    for folder in folders:
        folder_files=os.listdir(f'{path}\\{folder}')
        path_link=f'{path}\\{folder}'
        files=os.listdir(path_link)
        for file in files:
            link=f'{path_link}\\{file}'
            age=int(folder)
            result.append({
                'type':i,
                'image_name':file,
                'age':age,
                'link':link
            })
df=pd.json_normalize(result)

In [ ]:
for index,row in df.iterrows():
    if row['age']>19 and row['age']<=25:
        df.at[index,'label']=0
    elif row['age']>25 and row['age']<=30:
        df.at[index,'label']=1
    elif row['age']>30 and row['age']<=35:
        df.at[index,'label']=2
    elif row['age']>35 and row['age']<=40:
        df.at[index,'label']=3
    elif row['age']>40 and row['age']<=45:
        df.at[index,'label']=4
    else:
        df.at[index,'label']=5        

In [ ]:
print(df.type[6])
print(df.image_name[6])
print(df.age[6])
print(df.link[6])

In [ ]:
df=df.sample(frac=1).reset_index(drop=True) #sample or shuffle dataframe rows
train=df.copy()

In [ ]:
train.head()

In [ ]:
temp=train[:7000]
X=[]
count=0
for index,row in temp.iterrows():
    image = cv2.imread(row['link'])
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray=gray/255
    X.append(gray)
    count+=1
    print(f'{count}/{len(temp)}',end='\r')

In [ ]:
Y_train = temp["label"]
X_train=np.asarray(X)

In [ ]:
plt.figure(figsize=(15,7))
g = sns.countplot(Y_train, palette="icefire")
plt.title("Number of ages")


In [ ]:
img = X_train[6]
img = img.reshape((128,128,1))
plt.imshow(img,cmap='gray')
plt.title(Y_train[6])
plt.show()
print(temp['link'][6])

In [ ]:
Y_train = to_categorical(Y_train, num_classes = 51)
print("y_train shape: ",Y_train.shape)
X_train = X_train.reshape(-1,128,128,1).astype("float") 
print("x_train shape: ",X_train.shape)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size = 0.1, random_state=42)
print("x_train shape",X_train.shape,type(X_train))
print("x_test shape",X_test.shape,type(X_test))
print("y_train shape",Y_train.shape,type(Y_train))
print("y_test shape",Y_test.shape,type(Y_test))

In [ ]:
model = Sequential()
model.add(keras.layers.AveragePooling2D(3,3,input_shape=(128,128,1)))
model.add(Conv2D(128,3,activation='relu'))
model.add(Conv2D(64,3,activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(1))
model.add(Activation("softmax"))
model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [ ]:
model.fit(X_train,Y_train,epochs=5,batch_size=64)

In [ ]:
model.evaluate(X_test,Y_test)

In [ ]:
from kerastuner.tuners import RandomSearch
def build_model(hp):
    model=keras.Sequential()
    model.add(keras.layers.AveragePooling2D(6,3,input_shape=(128,128,1)))
    for i in range(hp.Int('Conv Layers',min_value=0,max_value=3)):
        model.add(keras.layers.Conv2D(hp.Choice(f'layer_{i}_filters',[16,32,64]),3,activation='relu'))     
    model.add(keras.layers.MaxPool2D(2,2))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128,activation='relu'))
    model.add(keras.layers.Dense(19,activation='softmax'))    
    
    model.compile(optimizer='adam',
             loss=keras.losses.SparseCategoricalCrossentropy(),
             metrics=['accuracy'])
    return model

In [ ]:
tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=6
                     )

In [ ]:
tuner.search(X_train,Y_train,validation_data=(X_val,Y_val),epochs=10,batch_size=64)

In [ ]:
best_model=tuner.get_best_models()[0]

In [ ]:
best_model.evaluate(X_val,Y_val)

In [ ]:
best_model.summary()